# Recommending restaurants to a user based off of their eating habits:

*This notebook is for the capstone project in IBM Data Science by Coursera.*

## Introduction / Business problem:

There are 29,560 restaurants in Los Angeles according to a count in 2018. That is a lot of potential restaurants to choose from ranging in cuisines, loation and quality. Finding a restaurant that you will like is a hard, time-consuming and often laborious task. Websites such as Yelp can be helpful showing you some of the best rated restaurants in your area. However, just because other people have enjoyed that restaurant does not necessarily mean you will enjoy it. That is why there is a need for a more personalized system of recommending a restaurant to the individual user. 

In this notebook I will do just that, creating a recommendation system based off of a user's previous reviews and ratings they have left at some of their favorite restaurants in Los Angeles. The recommender will take into account the cuisine, quality and location of the restaurants and personalize the recommendations to the user.

## Data:

The data that I will use to create the recommendation system will come from the *Foursquare API*. Two main datasets will be used. 

* A user's dataset in order to obtain the reviews, ratings, locations and restaurant history of the user. 
* A restaurants dataset that will contain the ultimate restaurants to be recommended to the user.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_rests = pd.DataFrame()
df_user = pd.DataFrame()